In [1]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
df = df.iloc[:300]

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
model = SentenceTransformer(model_name)

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
answer_llm = df.iloc[0].answer_llm

In [6]:
answer_llm_emb = model.encode(answer_llm)
answer_llm_emb[0]

-0.42244682

In [7]:
#Question 2

In [8]:
import numpy as np
from tqdm import tqdm

answer_embeddings_list = []
evaluations = []

for index, row in tqdm(df.iterrows()):
    answer_llm = row.answer_llm
    answer_gt = row.answer_orig

    answer_llm_embeddings = model.encode(answer_llm)
    answer_gt_embeddings = model.encode(answer_gt)

    answer_embeddings_list.append((answer_llm_embeddings, answer_gt_embeddings))
    score = np.dot(answer_llm_embeddings, answer_gt_embeddings)
    evaluations.append(score)
np.percentile(evaluations, 75)

300it [00:39,  7.51it/s]


31.674304962158203

In [ ]:
#question 3

In [ ]:
norm = np.sqrt((v * v).sum())
v_norm = v / norm

In [10]:
normalized_embeddings = []
for x in answer_embeddings_list:
    normalized_sublist = []
    for y in x:
        norm = np.linalg.norm(y)
        if norm == 0:
            normalized_sublist.append(y)
        else:
            normalized_sublist.append(y / norm)
    normalized_embeddings.append(normalized_sublist)

In [11]:
dot_products = []
for pair in normalized_embeddings:
    dot_product = np.dot(pair[0], pair[1])
    dot_products.append(dot_product)

In [12]:
np.percentile(dot_products, 75)

0.8362347483634949

In [ ]:
#question 4

In [13]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10].to_dict()
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [18]:
scores['rouge-1']['f']

0.45454544954545456

In [19]:
# question 5

In [21]:
np.mean([scores['rouge-1']['f'],scores['rouge-2']['f'],scores['rouge-l']['f']])

0.35490034990035496

In [26]:
#question 6

In [22]:
json_rouge = {
    'rouge_1': list(),
    'rouge_2': list(),
    'rouge_l': list()
}
for item in tqdm(df.itertuples()):
    scores = rouge_scorer.get_scores(item.answer_llm, item.answer_orig)[0]
    json_rouge['rouge_1'].append(scores['rouge-1']['f'])
    json_rouge['rouge_2'].append(scores['rouge-2']['f'])
    json_rouge['rouge_l'].append(scores['rouge-l']['f'])
df_rouge = pd.DataFrame(json_rouge)

300it [00:00, 741.79it/s]


In [25]:
df_rouge.describe()['rouge_2']['mean']

0.20696501983423318